# CI/CD Automation Using - Git source example


This example shows how to create a project YAML for CI/CD Automation from a Github source  - this process is equivalent for using tar, zip source 

After this example you will learn how to:
* Create a new MLRun project
* Connect a project to source
* Build a project YAML 

Install mlrun - if does not install use ``pip install mlrun==<mlrun server version>`` or ``sh align_mlrun.sh`` (our default mlrun installer - automatically install the server version)

In [1]:
import mlrun

MLRun provides you three option to create or loading a project:

1. [get_or_create_project](https://docs.mlrun.org/en/latest/api/mlrun.projects.html?highlight=get_or_create_project#mlrun.projects.get_or_create_project) - this method allows you to load a project from MLRun DB or optionally load it from a yaml/zip/tar/git template, or create/import if doesnt exist
2. [new_project](https://docs.mlrun.org/en/latest/api/mlrun.projects.html?highlight=new_project#mlrun.projects.new_project) - Create a new MLRun project, optionally load it from a yaml/zip/tar/git template.
3. [load_project](https://docs.mlrun.org/en/latest/api/mlrun.projects.html?highlight=load_project#mlrun.projects.load_project) - Load an MLRun project from yaml/zip/tar/git/dir or from MLRun db


On project creation MLRun create a light project YAML, for example: 
````
kind: project
metadata:
  name: default
  created: '2022-06-30T09:41:05.612000'
spec:
  functions: []
  workflows: []
  artifacts: []
  desired_state: online
status:
  state: online
````
For update project YAML use **projec.save()**

In [2]:
project_name_base = 'quick-start-tutorial'
project = mlrun.new_project(project_name_base,user_project=True,init_git=True)

The source define your source code this source need to include all your project files  - MLRun knows to point to the files in the source address and run the functions based on the source files

In [3]:
source = 'git://github.com/GiladShapira94/quick-start.git'

The pull_at_runtime flag will determine if the code is loaded in runtime or added to the image during build. the first (at runtime) option is better for debugging while the secound is better for production. Note that if you choose the 2nd option you'll need to build the function before run.

In [4]:
project.set_source(source=source,pull_at_runtime=True)

For functions definations use the [set_function](https://docs.mlrun.org/en/latest/api/mlrun.projects.html?highlight=set_function#mlrun.projects.MlrunProject.set_function) method.

The **set_function** method allow you to set the functions attributes in the project YAML, for example: 
function source (YAML, py, ipynb, function object) , name of the fucntion, function handler, function image and function kind.

````
project.set_function(
    name="training", handler="training.model_training",
    image="mlrun/mlrun", kind="job", with_repo=True,
)
````
> Set the with_repo=True to add the entire repo code into the destination container during build or run time. 

> When using with_repo=True the functions need to be deployed (function.deploy()) to build a container, unless you set project.spec.load_source_on_run=True which instructs MLRun to load the git/archive repo into the function container at run time and do not require a build (this is simpler when developing, for production it’s preferred to build the image with the code)


In [5]:
project.set_function(name='prep_data',handler='prep_data.prep_data',kind='job',image='mlrun/mlrun',with_repo=True)

In [6]:
project.set_function(name='train_iris',handler='train.train_iris',kind='job',image='mlrun/mlrun',with_repo=True)

**Important Note -** Remote functions as Serving need a function YAML as a source or function object
Before you creating fucntion YAML you need to the create a fucntion object you can do it with [code_to_function()](https://docs.mlrun.org/en/latest/api/mlrun.html?highlight=code_to_function#mlrun.code_to_function), [new_function()](https://docs.mlrun.org/en/latest/api/mlrun.run.html?highlight=new_function#mlrun.run.new_function).

After you creating a function object you can use the [export()](https://docs.mlrun.org/en/latest/api/mlrun.runtimes.html?highlight=export#mlrun.runtimes.BaseRuntime.export) method, For Example:
````
<function object>.export('./model_training.yaml')
````

````
project.set_function(
    func="training.yaml",name='training',with_repo=True,kind='serving')
````

In [7]:
serving = mlrun.new_function("serving", kind="serving", image="mlrun/mlrun")
serving.spec.default_class = 'ClassifierModel'
serving.add_model('model',model_path=project.get_artifact_uri('my_model'))



In [8]:
# save the function definition into a .yaml file and register it in the project
serving.export("serving.yaml")
project.set_function("serving.yaml", name="serving", with_repo=True)

> 2022-09-19 14:06:10,435 [info] function spec saved to path: serving.yaml


In [9]:
project.save()

### Done! 
**Now you have a project YAML for CI/CD Automation  - Later we will demostrate how to load a project and use this Project YAML**

**TIP -** After creating the project YAML Upload/Save it in the source